In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
MODEL = "llama3.1:latest"

In [2]:
question = "What kinds of things do i think about?"
document = "My favorite pet is a cat."

In [3]:
from langchain.prompts import PromptTemplate

template = """"
Answer the questions based on the given context below. If you cannot answer, reply with "I do not know".

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question = "Here is a question"))

"
Answer the questions based on the given context below. If you cannot answer, reply with "I do not know".

Context: Here is some context

Question: Here is a question




In [4]:
import torch
from torch import Tensor


In [5]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string(question, "cl100k_base")

9

In [7]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embedding_model = OllamaEmbeddings(model="llama3.1:latest")

# Embed the query and the document
query_result = embedding_model.embed_query(question)
document_result = embedding_model.embed_query(document)

# Check the length of the query result
len(query_result)



#model.invoke("tell me about ollama")

4096

In [8]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.6546856779155614


In [9]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("TAship letter.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'TAship letter.pdf', 'page': 0}, page_content='Kathmandu University\nSchool of Engineering\n[)ate: August I1,2024\n\'l\'o lvhonr it may conccrn,\nI arn writing [o reconllnend Mr. Anntol Acharya fbr any position or opportunity that rcquircs a clcclicated\nancl knowlcdgeable individual rvith a strong tbundation in cornputcr sciencc. l)urirrg his iirnc as a teachinu\nassistant at Kathnrarrdtr [Jniversity. Mr. Aclrarva has clcrnorrstratccl cxceptiolr:rl skills i1 both tcachirrg a,J\ntcchnical expertisc in coMP I l6: oh.ject-orientetl progranrming (c l r).\nl)rrrirlg ltis tenttrc as a tcaclting assistant (March 2024-ongoinu). cxpcctccl to last six rlonths. Mr. Aclrar,va\nhas cxhibited conrnrcttdatrle tcaching skills and a soliclgrasp olllrc su[-l.iect r]ratter. Ile was respopsible ltr\nleadilrg lab sessions. assistittg studcttts r,r\'ith prograrnnring coltcepts. arrcl cnsuring that the curriculunr is\ndelivered effectively. Ilis ability to simplily challcnging 

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(pages)
splits

[Document(metadata={'source': 'TAship letter.pdf', 'page': 0}, page_content="Kathmandu University\nSchool of Engineering\n[)ate: August I1,2024\n'l'o lvhonr it may conccrn,\nI arn writing [o reconllnend Mr. Anntol Acharya fbr any position or opportunity that rcquircs a clcclicated\nancl knowlcdgeable individual rvith a strong tbundation in cornputcr sciencc. l)urirrg his iirnc as a teachinu\nassistant at Kathnrarrdtr [Jniversity. Mr. Aclrarva has clcrnorrstratccl cxceptiolr:rl skills i1 both tcachirrg a,J\ntcchnical expertisc in coMP I l6: oh.ject-orientetl progranrming (c l r).\nl)rrrirlg ltis tenttrc as a tcaclting assistant (March 2024-ongoinu). cxpcctccl to last six rlonths. Mr. Aclrar,va\nhas cxhibited conrnrcttdatrle tcaching skills and a soliclgrasp olllrc su[-l.iect r]ratter. Ile was respopsible ltr"),
 Document(metadata={'source': 'TAship letter.pdf', 'page': 0}, page_content='has cxhibited conrnrcttdatrle tcaching skills and a soliclgrasp olllrc su[-l.iect r]ratter. Ile was r

In [18]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="mycollection",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="mycollection",
    embedding=embedding_model,
)

In [19]:
chain = prompt | model | parser
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question']}

In [20]:
import warnings
import pydantic



# Suppress the specific warning
warnings.filterwarnings("ignore", message="`pydantic.error_wrappers:ValidationError` has been moved")

# Monkey patching to maintain compatibility
import pydantic.error_wrappers
pydantic.error_wrappers.ValidationError = pydantic.ValidationError


In [21]:

from langchain_qdrant import QdrantVectorStore

vectorstore = QdrantVectorStore.from_documents(splits, embedding = embedding_model)

ResponseHandlingException: [WinError 10061] No connection could be made because the target machine actively refused it

In [26]:
retriever = vectorstore.as_retriever()
retriever.invoke("teaching assistant")

[Document(metadata={'source': 'TAship letter.pdf', 'page': 0}, page_content='has cxhibited conrnrcttdatrle tcaching skills and a soliclgrasp olllrc su[-l.iect r]ratter. Ile was respopsible ltr\nleadilrg lab sessions. assistittg studcttts r,r\'ith prograrnnring coltcepts. arrcl cnsuring that the curriculunr is\ndelivered effectively. Ilis ability to simplily challcnging tasks ancl his cleclication to ephapcing stuclent\nlcarning wcrc u\'ell-received by both students ancl faculty. He also denronstratecl strong orgarrizational\nskills by tttanagittg lab scssiotts cfflcicntlr,. grading assignrncnts pronrptly. ancl providirrg constructivc\nf\'eeclback that hclpecl students irnprovc tlrcii prograrninins r,biIiti"r.\nln additiorr to thc albrcntctttiortcd tasks. Mr. Acharva has also plal\'cd a crucial rolc irr thc tcclrlical'),
 Document(metadata={'source': 'TAship letter.pdf', 'page': 0}, page_content="Kathmandu University\nSchool of Engineering\n[)ate: August I1,2024\n'l'o lvhonr it may conc

In [27]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")

    
}
| prompt
| model
| parser
)

chain.invoke({"question": "What does Anmol teach as a teaching assistant?"})

'COMS (Computer Science) subjects, specifically object-oriented programming and C++.'